# Bank Note Authentication 

## Abstract:
    
Data were extracted from images that were taken for the evaluation of an authentication procedure for banknotes.

## Data Set Information:
    
Data were extracted from images that were taken from genuine and forged banknote-like specimens. For digitization, an 
industrial camera usually used for print inspection was used. The final images have 400x 400 pixels. Due to the object lens and
distance to the investigated object gray-scale pictures with a resolution of about 660 dpi were gained. Wavelet Transform tool
were used to extract features from images.

Source: https://archive.ics.uci.edu/ml/datasets/banknote+authentication

## Attribute Information:
    
variance - variance of Wavelet Transformed image (continuous)
skewness - skewness of Wavelet Transformed image (continuous)
curtosis - curtosis of Wavelet Transformed image (continuous)
entropy - entropy of image (continuous)
class - class (integer)

In [57]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
#let us start by importing the relevant libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
#reading the banknote dataset in a dataframe. 
columns = ["var","skewness","curtosis","entropy","class"]
df = pd.read_csv(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt",
    index_col=False, 
    names=columns
)

## George's Bonus
- How to properly hang function parameters?
- https://stackoverflow.com/questions/39457607/what-is-the-proper-level-of-indent-for-hanging-indent-with-type-hinting-in-pytho

In [60]:
df.head()

,var,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [61]:
df.shape

(1372, 5)

In [6]:
#Let us use seaborn distplot to analyze the distribution of our columns
f, ax = plt.subplots(1, 4, figsize=(10,3))
vis1 = sns.distplot(df["var"],bins=10, ax= ax[0])
vis2 = sns.distplot(df["skewness"],bins=10, ax=ax[1])
vis3 = sns.distplot(df["curtosis"],bins=10, ax= ax[2])
vis4 = sns.distplot(df["entropy"],bins=10, ax=ax[3])
f.savefig('subplot.png')


In [7]:
# now that we have a univariate distribution done, we can go ahead and determine the pairwise relationships in the dataset.
# we use the sns.pairplpot. Analyze the distrbutions.
# also, we add hue to see these distrbutions with the class col
sns.pairplot(df, hue="class")

In [63]:
# we separate the target variable (class) and save it in the y variable. Also the X contains the independant variables.
X = df.iloc[:,0:4].values
y = df.iloc[:,4].values

In [64]:
#splitting the data in test and train sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 10)

In [66]:
# scaling the data using the standard scaler
from sklearn.preprocessing import StandardScaler
X_train_sd = StandardScaler().fit_transform(X_train)
X_test_sd = StandardScaler().fit_transform(X_test)

## George's Challenge

- Any problem?

In [11]:
X_train_sd.T.shape

(4, 1097)

In [68]:
np.array([[1, 2, 3], [4, 5, 6]]).T

array([[1, 4],
       [2, 5],
       [3, 6]])

In [78]:
# generating the covariance matrix and the eigen values for the PCA analysis
cov_matrix = np.cov(X_train_sd.T) # the relevanat covariance matrix
print('Covariance Matrix \n%s' % cov_matrix)

#generating the eigen values and the eigen vectors
e_vals, e_vecs = np.linalg.eig(cov_matrix)
print('Eigenvectors \n%s' %e_vecs)
print('\nEigenvalues \n%s' %e_vals)

Covariance Matrix 
[[ 1.00091241  0.26106526 -0.37965807  0.2855868 ]
 [ 0.26106526  1.00091241 -0.78600078 -0.52370267]
 [-0.37965807 -0.78600078  1.00091241  0.32384024]
 [ 0.2855868  -0.52370267  0.32384024  1.00091241]]
Eigenvectors 
[[-0.24410388 -0.75674709  0.60295351  0.06476264]
 [-0.63914113  0.0457533  -0.11988157 -0.75830974]
 [ 0.61378454  0.15345344  0.50421613 -0.58778114]
 [ 0.3939295  -0.63379217 -0.60649943 -0.27438198]]

Eigenvalues 
[2.17821749 1.30130064 0.34425337 0.17987814]


## George's Bonus
- Python f-string
- https://realpython.com/python-f-strings/

In [70]:
print(f'Covariance Matrix \n {cov_matrix}')

Covariance Matrix 
 [[ 1.00091241  0.26106526 -0.37965807  0.2855868 ]
 [ 0.26106526  1.00091241 -0.78600078 -0.52370267]
 [-0.37965807 -0.78600078  1.00091241  0.32384024]
 [ 0.2855868  -0.52370267  0.32384024  1.00091241]]


In [72]:
name = 'John'
print(f'my name is {name}')

my name is John


In [79]:
# the "cumulative variance explained" analysis 
tot = sum(e_vals)
var_exp = [( i /tot ) * 100 for i in sorted(e_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print("Cumulative Variance Explained", cum_var_exp)

Cumulative Variance Explained [ 54.40579687  86.90865691  95.50714586 100.        ]


In [76]:
np.cumsum(e_vals/e_vals.sum())

array([0.54405797, 0.86908657, 0.95507146, 1.        ])

## George's Challenge
- Walk thru the code
- Easy and better way?

In [80]:
# Plotting the variance expalained by the principal components and the cumulative variance explained.
plt.figure(figsize=(10 , 5))
plt.bar(range(1, e_vals.size + 1), var_exp, alpha = 0.5, align = 'center', label = 'Individual explained variance')
plt.step(range(1, e_vals.size + 1), cum_var_exp, where='mid', label = 'Cumulative explained variance')
plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.legend(loc = 'best')
plt.tight_layout()
plt.show()

In [81]:
e_vals

array([2.17821749, 1.30130064, 0.34425337, 0.17987814])

In [82]:
e_vecs

array([[-0.24410388, -0.75674709,  0.60295351,  0.06476264],
       [-0.63914113,  0.0457533 , -0.11988157, -0.75830974],
       [ 0.61378454,  0.15345344,  0.50421613, -0.58778114],
       [ 0.3939295 , -0.63379217, -0.60649943, -0.27438198]])

In [83]:
eigen_pairs = [(np.abs(e_vals[i]), e_vecs[:,i]) for i in range(len(e_vals))]
eigen_pairs.sort(reverse=True)
eigen_pairs[:5]

[(2.1782174876819482,
  array([-0.24410388, -0.63914113,  0.61378454,  0.3939295 ])),
 (1.3013006373731917,
  array([-0.75674709,  0.0457533 ,  0.15345344, -0.63379217])),
 (0.34425337148288904,
  array([ 0.60295351, -0.11988157,  0.50421613, -0.60649943])),
 (0.17987813849846412,
  array([ 0.06476264, -0.75830974, -0.58778114, -0.27438198]))]

In [40]:
np.hstack(
    (eigen_pairs[0][1].reshape(4,1), 
     eigen_pairs[1][1].reshape(4,1))
)

array([[-0.24410388, -0.75674709],
       [-0.63914113,  0.0457533 ],
       [ 0.61378454,  0.15345344],
       [ 0.3939295 , -0.63379217]])

In [84]:
# generating dimensionally reduced datasets
w = np.hstack(
    (eigen_pairs[0][1].reshape(4,1), 
     eigen_pairs[1][1].reshape(4,1))
)
print('Matrix W:\n', w)
X_sd_pca = X_train_sd.dot(w)
X_test_sd_pca = X_test_sd.dot(w)

Matrix W:
 [[-0.24410388 -0.75674709]
 [-0.63914113  0.0457533 ]
 [ 0.61378454  0.15345344]
 [ 0.3939295  -0.63379217]]


In [85]:
X_train_sd.shape, w.shape, X_sd_pca.shape, X_test_sd_pca.shape

((1097, 4), (4, 2), (1097, 2), (275, 2))

## George's Challenge
What is happening??

## George's Tip
Use `.T` to transpose

In [55]:
X_test_sd_pca[:5, :]

array([[ 1.87240322, -0.47302236],
       [ 0.30717397, -0.73655748],
       [-1.39781662, -0.20075128],
       [ 0.41406841, -0.87916121],
       [ 0.00214918, -0.70804641]])

In [56]:
e_vecs[:, :2].T.dot(X_test_sd.T).T[:5, :]

array([[ 1.87240322, -0.47302236],
       [ 0.30717397, -0.73655748],
       [-1.39781662, -0.20075128],
       [ 0.41406841, -0.87916121],
       [ 0.00214918, -0.70804641]])

## George's Bonus
- Matrix multiplication
- More PCA!
 - https://machinelearningmastery.com/calculate-principal-component-analysis-scratch-python/
 - https://towardsdatascience.com/the-mathematics-behind-principal-component-analysis-fff2d7f4b643
 - https://builtin.com/data-science/step-step-explanation-principal-component-analysis

In [94]:
a1 = np.array([[1, 2, 3]])
a1

array([[1, 2, 3]])

In [95]:
a1.shape

(1, 3)

In [90]:
a2 = np.array([[3, 4, 5, 5], [6, 7, 8, 6], [1, 1, 1, 1]])
a2

array([[3, 4, 5, 5],
       [6, 7, 8, 6],
       [1, 1, 1, 1]])

In [91]:
a2.shape

(3, 4)

In [92]:
a3 = np.array([[3, 4]])
a3

array([[3, 4]])

In [45]:
a1.shape, a2.shape

((1, 2), (3, 4))

In [97]:
a1.dot(a2).shape

(1, 4)

In [60]:
np.multiply(a1, a3)

array([[3, 8]])

### We will use Logistic regression, RandomForest and AdaBoost

In [98]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_sd, y_train)
print ('Before PCA score', model.score(X_test_sd, y_test))

model.fit(X_sd_pca, y_train)
print ('After PCA score', model.score(X_test_sd_pca, y_test))


Before PCA score 0.9854545454545455
After PCA score 0.7709090909090909


In [99]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train_sd, y_train)
print ('Before PCA score', clf.score(X_test_sd, y_test))

clf.fit(X_sd_pca, y_train)
print ('After PCA score', clf.score(X_test_sd_pca, y_test))


Before PCA score 0.9890909090909091
After PCA score 0.8690909090909091


In [100]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier()
model.fit(X_train_sd, y_train)
print ('Before PCA score', model.score(X_test_sd, y_test))

model.fit(X_sd_pca, y_train)
print ('After PCA score', model.score(X_test_sd_pca, y_test))

Before PCA score 0.9963636363636363
After PCA score 0.8472727272727273


### In the given dataset we trained models with the orginal and dimensionally reduced datsets. The effects of PCA can be clearly appreciated on a fairly large datsaset. The learners are encouraged to try the above with various large datsets out there.